# Introduction

This notebook demonstrates typical initial steps to exploring phenotype distributions. It has been written to be interactive, allowing you to make choices as you go.

## Data disclaimer
----

All data in this notebook (and this workspace) are publicly available thanks to the effort of many dedicated individuals: 

- Genotype and some phenotypic data were produced by the [1000 Genomes Project (phase 3)](https://www.internationalgenome.org/)

- Individual phenotypes were modeling using the [GCTA software](cnsgenomics.com/software/gcta) and variant-level summary statistics from [MAGIC](https://www.magicinvestigators.org/), the [GIANT Consortium](https://portals.broadinstitute.org/collaboration/giant/index.php/Main_Page), the [UK Biobank](https://www.ukbiobank.ac.uk/), and the [MVP](https://www.research.va.gov/mvp/)  

Phenotypes were modeled to reflect the actual genetic architecture of these complex traits as closely as possible. Most single variant association results should correspond well to published GWAS, but others may not. **Results produced from these data should not be taken as representing real, replicable genetic associations. These data are provided for demonstration and training purposes only.**

## Load Python packages
----

* **Pandas & Numpy** - packages for data analysis  
* **Pprint** - for pretty printing  
* **Matplotlib & Seaborn** - for plotting  

To see the entire list of Python packages, click the purple arrow to the right below.

In [ ]:
%%capture 
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import uuid
import urllib

In [ ]:
# Paste in the JWT token obtained via Azure B2C
token = ""
# Paste snapshot ID generated from the AzureY1Demo notebook
snapshot_id = ""

# Load phenotypes 

Phenotypic data for each individual in the study are stored in Azure TDR. To analyze inside this notebook, we have to explicitly load the data in our notebook environment. 

## Retrieve TDR snapshot and copy parquet data to the notebook

Note: this duplicates some functionality from the `AzureY1Demo` notebook. Omit this step if the parquet file has already been copied to the notebook disk.

In [ ]:
from data_repo_client import Configuration
from data_repo_client import ApiClient
from data_repo_client import SnapshotsApi

# Set up configuration
config = Configuration()
config.host="https://jade-nm.datarepo-dev.broadinstitute.org/"
config.access_token = token
api_client = ApiClient(configuration=config)
api_client.client_side_validation = False
local_parquet_dir = "/tmp/az"

# Create required API Clients
snapshots_api = SnapshotsApi(api_client=api_client)

# Copy parquet data from TDR
snapshot = snapshots_api.retrieve_snapshot(snapshot_id, include=["ACCESS_INFORMATION"])
table = next(iter(snapshot.access_information.parquet.tables), lambda t: t.name == "demo_pheno_data")
os.system("rm -r %s/%s.parquet" % (local_parquet_dir, table.name))
os.system("azcopy cp '%s?%s' '%s' --recursive" % (table.url, table.sas_token, local_parquet_dir))

## Load phenotype data

Load phenotype data into a pandas dataframe. The columns correspond to:  

* **sample:** a unique label for each individual sample in our dataset
* **age:** numerical age of the individual at the time of each phenotype measure
* **ancestry:** superpopulation group of each individual
  * AFR: African
  * AMR: Ad Mixed American
  * EAS: East Asian
  * EUR: European
  * SAS: South Asian
* **bmi:** body mass index
* **fg:** fasting glucose
* **fi:** fasting insulin
* **hdl:** high density lipoprotein
* **height:** standing height
* **ldl:** low density lipoprotein
* **population:** population of each sample, see [1000 Genomes description](https://www.internationalgenome.org/category/population/)
* **sex:** biological sex
* **tc:** total cholesteral
* **tg:** total triglycerides
* **whr:** waist-to-hip ratio

In [ ]:
samples = pd.read_parquet("%s/%s.parquet" % (local_parquet_dir, table.name))
samples = samples.drop(columns=["datarepo_row_id"])
samples.head(10)

# Examine phenotype data
----

Let's take a look at the phenotype distributions. In a GWAS - and statistical genetics more generally - we should always be on the lookout for correlations within our dataset. Correlations between phenotypic values can confound our analysis, leading to results that may not represent true genetic associations with our traits. Exploring these relationships may help in choosing a reasonable set of covariates to model.    

We've included a number of plotting functions below to make this as easy as possible. Feel free to modify - or write your own functions - as you explore the data. 


## Goals of this section
----
    
1. Visualize the distribution of phenotype values  
    - Within each continuous trait (using the kdplot function)  
    - Within each continuous trait, organized by dichotomous data (ex: the distribution of BMI in each ancestry group - using the boxPlot function)  
    - Between two continuous traits (with the bivariateDistributionPlot function)
2. Determine whether trait distributions follow patterns we might expect

## Code for plotting functions
    
The next code block defines the plotting functions. Because running the plotting functions doesn't require understanding the syntax of the code, we have collapsed this block. Feel free to uncollapse (click the purple arrow at the right) if you're interested in all the details!

In [ ]:
# Define functions to easily plot phenotypes
plt.rcParams["figure.figsize"] = [6,4]
    
# Visualize distribution with each continuous trait
def kdPlot(data, var):
    sns.set_style("whitegrid")
    sns.set_context("poster", 
                    font_scale = 0.9, 
                    rc={"grid.linewidth": 0.6, 'lines.linewidth': 1.6})
    sns.distplot(data[(var)])
    
# Visualize the distribution between two continuous traits
def bivariateDistributionPlot(data, var1, var2, kind = "scatter"):
    with sns.axes_style("whitegrid"):
        jplot = sns.jointplot(x = data[var1], y = data[var2], kind=kind, color="k", s=1)
        jplot.set_axis_labels(var1, var2)
        
# Visualize within each continuous trait, organized by dichotomous data
def boxPlot(data, catagorical_var, continuous_var, color_by = None, force_x = False, force_color = False):
    make_plot = True
    if len(data[catagorical_var].unique().tolist()) > 10 and force_x is not True:
        make_plot = False
        print("catagorical_var must be catagorical. If you insist on using these x values, set force_x = True.")
    if color_by is not None:
        if len(data[color_by].unique()) > 5 and force_color is not True:
            make_plot = False
            print("color_by column must be catagorical. If you insist on using these values, set force_color = True.")
    
    if (make_plot is True):
        sns.set_style("whitegrid")
        sns.set_context("poster", 
                        font_scale = 0.7, 
                        rc={"grid.linewidth": 0.6, 'lines.linewidth': 1.6})
        sns.boxplot(x = catagorical_var, 
                    y = continuous_var, 
                    hue = color_by, 
                    data = data, 
                    palette = ["#275F9A", "#A2C353"],
                    saturation = 1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## Generating distribution plots

<img src="https://raw.githubusercontent.com/tmajaria/ashg_2019_workshop/master/ldl_kdplot.png" align="left" width="20%">

***Univariate distributions*** are easily visualized in histograms or density plots. We provide a function (<font color='red'>kdplot</font>) that will generate both types of plots, overlayed in a single figure. A continuously-valued variable corresponding to a column in the phenotype dataframe should be used as input, *ldl* in this example. The function is called with the following syntax:

```python
kdPlot(samples, var = "ldl")
```

<img src="https://raw.githubusercontent.com/tmajaria/ashg_2019_workshop/master/whr_hdl_bivariateDistributionPlot.png" align="left" width="20%"> 

***Bivariate distributions*** can be visualized using a scatterplot. Use the function <font color='red'>bivariateDistributionPlot</font> to visualize two continuously values variables. The *type* argument determines the type of plot generated and can be one of: "scatter", "reg", "resid", "kde", and "hex".

```python
bivariateDistributionPlot(samples, var1 = "hdl", var2 = "whr", kind = "scatter")
```

<img src="https://raw.githubusercontent.com/tmajaria/ashg_2019_workshop/master/height_ancestry_boxPlot.png" align="left" width="20%">

***Boxplots*** can be used to further interogate relationships between continuous and categorical variables, using the <font color='red'>boxPlot</font> function. Distributions can be further subset by sex (by setting *color_by* = "sex").

```python
boxPlot(samples, catagorical_var = "population", continuous_var = "height", color_by = "sex")
```

### Exercise: Univariate distributions

Use the code cells below to plot the distribution of single variables of your choice (such as ldl or bmi). You may need to refer to section 3.2 above for the list of variables and to section 4.1 for the plotting syntax. 

In [ ]:
kdPlot(samples, var = "ldl")

### Exercise: Bivariate distributions

Generate scatter plots with different combinations of variables. Think about what you would expect versus what you see in the plot. You may need to refer to 3.2 for the list of variables and to section 4.1 for the plotting syntax. 

In [ ]:
bivariateDistributionPlot(samples, var1 = "bmi_baseline", var2 = "ldl", kind = "scatter")

### Exercise: Boxplots 

Boxplots show relationships between continuous and categorical variables. Use the code cell below to generate a boxplot showing the relationship between height and ancestry. Try adding sex as another delineator with the `color_by` argument. You may need to refer to 3.2 for the list of variables and to section 4.1 for the plotting syntax.

In the next code block, try a boxplot with variables of your choice.

In [ ]:
boxPlot(samples, catagorical_var = "population", continuous_var = "height_baseline", color_by = "sex", force_x = True)

# Look at IGV
----

Just for fun, let's use the `igv-jupyter` notebook extension to look at genotype information for 1 sample.


In [ ]:
# Get the DRS URLs from the sample table
bam_file = samples.at[0, 'bam_file']
bam_file_index = samples.at[0, 'bam_file_index']

print(f'bam_file: {bam_file}')
print(f'bam_file_index: {bam_file_index}')

In [ ]:
# Use terra-notebook-utils to resolve the DRS
# TODO doesn't work, complains about gcloud at import time

#from terra_notebook_utils import drs
#drs.info(bam_file)
#drs.access(bam_file)

In [ ]:
# Call Martha to resolve the DRS
# TODO this succeeds but doesn't return the access_url - why?

#import requests
#endpoint = 'https://us-central1-broad-dsde-dev.cloudfunctions.net/martha_v3'
#data = {"url": bam_file}
#headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

#requests.post(endpoint, data=json.dumps(data), headers=headers).json()

In [ ]:
# Just resolve the DRS directly with TDR
from data_repo_client import DataRepositoryServiceApi
drs_api = DataRepositoryServiceApi(api_client=api_client)

def get_access_url(drs):
    drs_id = drs[drs.rfind("/")+1:]
    drs_object = drs_api.get_object(drs_id)
    drs_access = drs_api.get_access_url(drs_id, drs_object.access_methods[0].access_id)
    return drs_access.url

bam_file_access_url = get_access_url(bam_file)
bam_file_index_access_url = get_access_url(bam_file_index)

print(f"bam_file access: {bam_file_access_url}")
print(f"bam_file_index access: {bam_file_index_access_url}")

In [ ]:
# Copy the files to the notebook disk
! wget -O HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20101123.bam "{bam_file_access_url}"
! wget -O HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20101123.bam.bai "{bam_file_index_access_url}"

In [ ]:
# Load up IGV
import igv

b = igv.Browser({"genome": "hg38"})

b.load_track(
  {
    "name": "HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20101123.bam",
    "url": "HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20101123.bam",
    "indexURL": "HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20101123.bam.bai",
    "format": "bam",
    "type": "alignment"
  }
)

b.search('chr20:32,214,217-32,229,950')

b.show()